<a href="https://colab.research.google.com/github/hammer1234567/Helmets-Masks/blob/master/Mask%26Helmet_models_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
os.chdir('/content/drive/My Drive/google_project/interview_data')
!ls

 annotations			      model_helmet.h5
 annotations.xml		      model_helmet.json
 darknet			      model_mask.h5
 dataset.csv			      model_mask.json
 final				      MyFile.txt
'Integrating_yolo&NN_models.ipynb'    YOLO_Training.ipynb
 interview_data			      Yoo_Data_Generation.ipynb
'Mask&Helmet_models_training.ipynb'


In [2]:
import pandas as pd
data = pd.read_csv('dataset.csv')
data.head()

,image,mask,helmet
0,final/0/0.jpg,yes,yes
1,final/0/1.jpg,no,yes
2,final/1/0.jpg,yes,no
3,final/2/0.jpg,no,yes
4,final/2/1.jpg,invisible,no


# Training model for mask detection on heads detected by yolov3 model


In [0]:
import tensorflow
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [4]:
datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        validation_split = 0.2)

train_data = datagen.flow_from_dataframe(
    dataframe=data,
    directory = './',
    x_col = 'image',
    y_col="mask",
    target_size=(64, 64),
    batch_size = 32,
    shuffle = True,
    class_mode = "categorical",
    subset = 'training'
)

valid_data = datagen.flow_from_dataframe(
    dataframe = data,
    directory = './',
    x_col = 'image',
    y_col = "mask",
    target_size = (64, 64),
    batch_size =32,
    shuffle = True,
    class_mode = "categorical",
    subset = 'validation'
)

/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/dataframe_iterator.py:282: UserWarning: Found 1 invalid image filename(s) in x_col="image". These filename(s) will be ignored.
  .format(n_invalid, x_col)


Found 1075 validated image filenames belonging to 4 classes.
Found 268 validated image filenames belonging to 4 classes.


In [0]:
import tensorflow as tf
from tensorflow.compat.v1.layers import MaxPooling2D
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import InputLayer
# from keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense

In [6]:
vgg16_model = tensorflow.keras.applications.vgg16.VGG16()

553467904/553467096 [==============================] - 6s 0us/step


In [0]:
model_mask = Sequential()
model_mask.add( InputLayer(input_shape=(64, 64, 3), name='image_input') )

In [8]:
model_mask.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________


In [0]:
for layer in vgg16_model.layers[1:-4]:
  model_mask.add(layer)

In [10]:
model_mask.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        multiple                  1792      
_________________________________________________________________
block1_conv2 (Conv2D)        multiple                  36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   multiple                  0         
_________________________________________________________________
block2_conv1 (Conv2D)        multiple                  73856     
_________________________________________________________________
block2_conv2 (Conv2D)        multiple                  147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   multiple                  0         
_________________________________________________________________
block3_conv1 (Conv2D)        multiple                  2

In [0]:
for layer in model_mask.layers:
  layer.trainable = False

In [0]:
model_mask.add(Conv2D(64, (2, 2), padding='same'))
model_mask.add(Activation('relu'))
model_mask.add(MaxPool2D(pool_size=(2, 2), strides = (1,1), padding='same'))

model_mask.add(Conv2D(64, (2, 2), padding='same'))
model_mask.add(Activation('relu'))
model_mask.add(MaxPool2D(pool_size=(2, 2), strides = (1,1), padding='same'))

In [0]:
model_mask.add(Flatten()) 

In [0]:
model_mask.add(Dense(128))
model_mask.add(Activation('relu'))
model_mask.add(Dense(64))
model_mask.add(Activation('relu'))
model_mask.add(Dropout(0.25))
model_mask.add(Dense(4))
model_mask.add(Activation('softmax'))

In [15]:
model_mask.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        multiple                  1792      
_________________________________________________________________
block1_conv2 (Conv2D)        multiple                  36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   multiple                  0         
_________________________________________________________________
block2_conv1 (Conv2D)        multiple                  73856     
_________________________________________________________________
block2_conv2 (Conv2D)        multiple                  147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   multiple                  0         
_________________________________________________________________
block3_conv1 (Conv2D)        multiple                  2

In [16]:
from keras import metrics
model_mask.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.SGD(
                  learning_rate=0.01, momentum=0.0, nesterov=False, name='SGD'),
              metrics=[metrics.categorical_accuracy])

Using TensorFlow backend.


In [17]:
model_mask.fit(
        train_data,
        epochs=100,
        validation_data=valid_data,
        )

Epoch 1/100
34/34 [==============================] - 1158s 34s/step - loss: 1.2589 - categorical_accuracy: 0.3340 - val_loss: 1.1309 - val_categorical_accuracy: 0.4627
Epoch 2/100
34/34 [==============================] - 3s 97ms/step - loss: 1.1503 - categorical_accuracy: 0.3693 - val_loss: 1.1226 - val_categorical_accuracy: 0.3507
Epoch 3/100
34/34 [==============================] - 3s 95ms/step - loss: 1.1190 - categorical_accuracy: 0.3907 - val_loss: 1.1028 - val_categorical_accuracy: 0.3545
Epoch 4/100
34/34 [==============================] - 3s 96ms/step - loss: 1.0659 - categorical_accuracy: 0.4335 - val_loss: 1.1007 - val_categorical_accuracy: 0.3321
Epoch 5/100
34/34 [==============================] - 3s 96ms/step - loss: 1.0663 - categorical_accuracy: 0.4605 - val_loss: 1.0795 - val_categorical_accuracy: 0.3470
Epoch 6/100
34/34 [==============================] - 3s 96ms/step - loss: 1.0454 - categorical_accuracy: 0.4586 - val_loss: 1.0497 - val_categorical_accuracy: 0.4328
Ep

In [19]:
labels_mask = (train_data.class_indices)
labels_mask = dict((v,k) for k,v in labels_mask.items())
labels_mask

{0: 'invisible', 1: 'no', 2: 'wrong', 3: 'yes'}

In [0]:
model_mask.save_weights("model_mask.h5")

In [0]:
model_mask_json = model_mask.to_json()
with open("model_mask.json", "w") as json_file:
    json_file.write(model_mask_json)

# Training model for Helmet detection on heads detected from Yolo V3 model

In [0]:
import tensorflow
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [25]:
datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        validation_split = 0.2)

train_data = datagen.flow_from_dataframe(
    dataframe=data,
    directory = './',
    x_col = 'image',
    y_col="helmet",
    target_size=(64, 64),
    batch_size = 32,
    shuffle = True,
    class_mode = "binary",
    subset = 'training'
)

valid_data = datagen.flow_from_dataframe(
    dataframe = data,
    directory = './',
    x_col = 'image',
    y_col = "helmet",
    target_size = (64, 64),
    batch_size =32,
    shuffle = True,
    class_mode = "binary",
    subset = 'validation'
)

/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/dataframe_iterator.py:282: UserWarning: Found 1 invalid image filename(s) in x_col="image". These filename(s) will be ignored.
  .format(n_invalid, x_col)


Found 1075 validated image filenames belonging to 2 classes.
Found 268 validated image filenames belonging to 2 classes.


In [0]:
import tensorflow as tf
from tensorflow.compat.v1.layers import MaxPooling2D
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import InputLayer
# from keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense

In [0]:
model_helmet = Sequential()
model_helmet.add( InputLayer(input_shape=(64, 64, 3), name='image_input') )

In [0]:
for layer in vgg16_model.layers[1:-4]:
  model_helmet.add(layer)

In [29]:
model_helmet.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        multiple                  1792      
_________________________________________________________________
block1_conv2 (Conv2D)        multiple                  36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   multiple                  0         
_________________________________________________________________
block2_conv1 (Conv2D)        multiple                  73856     
_________________________________________________________________
block2_conv2 (Conv2D)        multiple                  147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   multiple                  0         
_________________________________________________________________
block3_conv1 (Conv2D)        multiple                 

In [0]:
for layer in model_helmet.layers:
  layer.trainable = False

In [0]:
model_helmet.add(Conv2D(64, (2, 2), padding='same'))
model_helmet.add(Activation('relu'))
model_helmet.add(MaxPool2D(pool_size=(2, 2), strides = (1,1), padding='same'))

model_helmet.add(Conv2D(64, (2, 2), padding='same'))
model_helmet.add(Activation('relu'))
model_helmet.add(MaxPool2D(pool_size=(2, 2), strides = (1,1), padding='same'))

In [0]:
model_helmet.add(Flatten()) 

In [0]:
model_helmet.add(Dense(128))
model_helmet.add(Activation('relu'))
model_helmet.add(Dense(64))
model_helmet.add(Activation('relu'))
model_helmet.add(Dropout(0.25))
model_helmet.add(Dense(1))
model_helmet.add(Activation('sigmoid'))

In [34]:
model_helmet.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        multiple                  1792      
_________________________________________________________________
block1_conv2 (Conv2D)        multiple                  36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   multiple                  0         
_________________________________________________________________
block2_conv1 (Conv2D)        multiple                  73856     
_________________________________________________________________
block2_conv2 (Conv2D)        multiple                  147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   multiple                  0         
_________________________________________________________________
block3_conv1 (Conv2D)        multiple                 

In [0]:
from keras import metrics
model_helmet.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.SGD(
                  learning_rate=0.01, momentum=0.0, nesterov=False, name='SGD'),
              metrics=[metrics.binary_accuracy])

In [36]:
model_helmet.fit(
        train_data,
        epochs=100,
        validation_data=valid_data,
        )

Epoch 1/100
34/34 [==============================] - 3s 97ms/step - loss: 0.6245 - binary_accuracy: 0.6837 - val_loss: 0.6132 - val_binary_accuracy: 0.6828
Epoch 2/100
34/34 [==============================] - 3s 91ms/step - loss: 0.6083 - binary_accuracy: 0.6949 - val_loss: 0.6040 - val_binary_accuracy: 0.6828
Epoch 3/100
34/34 [==============================] - 3s 92ms/step - loss: 0.5966 - binary_accuracy: 0.6949 - val_loss: 0.5909 - val_binary_accuracy: 0.6828
Epoch 4/100
34/34 [==============================] - 3s 92ms/step - loss: 0.5819 - binary_accuracy: 0.6949 - val_loss: 0.5780 - val_binary_accuracy: 0.6828
Epoch 5/100
34/34 [==============================] - 3s 91ms/step - loss: 0.5711 - binary_accuracy: 0.6967 - val_loss: 0.5538 - val_binary_accuracy: 0.6940
Epoch 6/100
34/34 [==============================] - 3s 91ms/step - loss: 0.5671 - binary_accuracy: 0.7014 - val_loss: 0.5563 - val_binary_accuracy: 0.6828
Epoch 7/100
34/34 [==============================] - 3s 92ms/ste

In [37]:
labels_helmet = (train_data.class_indices)
labels_helmet = dict((v,k) for k,v in labels_helmet.items())
labels_helmet

{0: 'no', 1: 'yes'}

In [0]:
model_helmet.save_weights("model_helmet.h5")

In [0]:
model_helmet_json = model_helmet.to_json()
with open("model_helmet.json", "w") as json_file:
    json_file.write(model_helmet_json)